In [100]:
# Dependencies and Setup
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
import time

In [101]:
#Insert splinter path to chromedriver
executable_path = {'executable_path': r"C:\chromedriver"}
browser = Browser('chrome', **executable_path, headless=False)

In [102]:
#All of these sites must be scraped
news_url = 'https://mars.nasa.gov/news/'
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
facts_url = 'http://space-facts.com/mars/'
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

## NASA MARS NEWS

In [64]:
#access NASA Mars page using splinter
browser.visit(news_url)
time.sleep(5)

#html object
news_html = browser.html
#create beautiful soup object to parse
soup = bs(news_html, 'html.parser')

In [75]:
#find title and p elements using splinter after looking in inspector
titles = soup.find_all('div', class_='content_title')
p_elements = soup.find_all('div', class_='article_teaser_body')
news_title = titles[1].text
news_p=p_elements[0].text
print(news_title)
print("----------------------")
print(news_p)

NASA Moves Forward With Campaign to Return Mars Samples to Earth
----------------------
During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.


## MARS IMAGE

In [76]:
#tell splinter to visit image url
browser.visit(image_url)
time.sleep(5)
#html object
img_html = browser.html
img_soup = bs(img_html, 'html.parser')

In [77]:
#get url of full image from inside style tag
featured_image = img_soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Display url of the full image
featured_image_url = f"https://www.jpl.nasa.gov{featured_image}"
print("JPL Featured Space Image")
print("-----------------------------------------")
print(featured_image_url)

JPL Featured Space Image
-----------------------------------------
https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19347-1920x1200.jpg


# MARS FACTS

In [78]:
# use pandas to scrape and create df
fact_list = pd.read_html(facts_url)

In [85]:
#figure out how many tables are on page
#len(fact_list)
#navigate to desired table
fact_df = fact_list[0].rename(columns={0:"Descriptor", 1: "Mars"})
fact_df

,Descriptor,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [88]:
#save df into html table format
mars_html_facts = fact_df.to_html()

# MARS HEMISPHERE

In [103]:
#use splinter to visit hemispheres site
browser.visit(hemispheres_url)

#main url for loop
hemisphere_base_url = 'https://astrogeology.usgs.gov'
time.sleep(5)

In [104]:
#HTML object
html_hemisphere = browser.html

#start parsing with BS
soup = bs(html_hemisphere, 'html.parser')

#get all info about hemispheres by scraping item classes
astropedia = soup.find_all('div', class_='description')

In [106]:
#create list to populate using for loop for image urls
hemisphere_image_url =[]

#loop which gathers hemisphere info
for item in astropedia:
    #Take name of hemisphere and clean it
    item_name=str(item.find('a').text).replace(" Enhanced","")
    
    #create new link to download full quality image
    url_add=item.find('a')['href']
    full_url=hemisphere_base_url+url_add
    
    #click on hemisphere page
    browser.visit(full_url)
    time.sleep(5)
    link=browser.html
    
    #parse through second page
    souplink=bs(link,'html.parser')
    #from second site, hone in on link using class = wide-image
    item_image=hemisphere_base_url+souplink.find('img',class_="wide-image")['src']
    #append links to list in python dictionary form
    hemisphere_image_url.append({"title":item_name, "img_url":item_image})

In [107]:
print(hemisphere_image_url)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [108]:
#quit splinter browsing after completing scraping tasks
browser.quit

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x000001629BF365F8>>